## Gather Two Years of Data

In [10]:
!pip install gdax
!pip install utils

You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [78]:
import gdax
from datetime import datetime, timedelta 


In [88]:
def save_historical_data_plot(coin_name):
    public_client = gdax.PublicClient()


    end = datetime.today()
    start = end - timedelta(days=100)
    
    #do 8 loops of 100 days to capture two years of data
    prices = []

    for i in range(0,8):
        dataset = public_client.get_product_historic_rates(
            coin_name,
            start=start.isoformat(),
            end=end.isoformat(),
            granularity=24*60*60
        )
        for d in dataset:
            prices.append((d[1] + d[2])/2)
        
        
        end = start
        start = end - timedelta(days=100)


    return prices

In [89]:
bitcoin = save_historical_data_plot("BTC-USD")
len(bitcoin)

799

In [90]:
etherum = save_historical_data_plot("ETH-USD")
len(etherum)

731

## Basic Linear Regression

In [33]:
from scipy import stats
import numpy as np

In [100]:
def linear(coin):
    slope, intercept, r_value, p_value, std_err = stats.linregress(coin, range(0, len(coin)))
    return slope, intercept, r_value, p_value, std_err

### Bitcoin

In [101]:
btc_reg = linear(bitcoin)
btc_reg

(-0.04418099045499821,
 563.8424507234893,
 -0.8102949982882088,
 3.712433529723488e-187,
 0.001131819560737972)

In [102]:
#y = mx + b
bitcoin_prediction = btc_reg[0]*(len(bitcoin)+1) + btc_reg[1]
bitcoin_prediction

528.4976583594907

### Etherum

In [103]:
eth_reg = linear(etherum)
eth_reg

(-0.5700547071491155,
 513.0097800936542,
 -0.834011986584604,
 1.8925762856668771e-190,
 0.013967540137694476)

In [104]:
#y = mx + b
eth_prediction = eth_reg[0]*(len(etherum)+1) + eth_reg[1]
eth_prediction

95.72973446050162

## Ordinary Least Square

In [47]:
!pip install git+git://github.com/statsmodels/statsmodels

  Cloning git://github.com/statsmodels/statsmodels to c:\users\latif\appdata\local\temp\pip-5conl09e-build
  Found existing installation: statsmodels 0.8.0
    Uninstalling statsmodels-0.8.0:
      Successfully uninstalled statsmodels-0.8.0
  Running setup.py install for statsmodels: started
    Running setup.py install for statsmodels: finished with status 'error'
    Complete output from command C:\Users\latif\Anaconda3\python.exe -u -c "import setuptools, tokenize;__file__='C:\\Users\\latif\\AppData\\Local\\Temp\\pip-5conl09e-build\\setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" install --record C:\Users\latif\AppData\Local\Temp\pip-vho6wepi-record\install-record.txt --single-version-externally-managed --compile:
    Cythonizing sources
    Processing statsmodels\nonparametric\linbin.pyx
    Processing statsmodels\nonparametric\_smoothers_lowess.pyx
    Processing statsmodels\tsa\kalmanf\kalm

Command "C:\Users\latif\Anaconda3\python.exe -u -c "import setuptools, tokenize;__file__='C:\\Users\\latif\\AppData\\Local\\Temp\\pip-5conl09e-build\\setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" install --record C:\Users\latif\AppData\Local\Temp\pip-vho6wepi-record\install-record.txt --single-version-externally-managed --compile" failed with error code 1 in C:\Users\latif\AppData\Local\Temp\pip-5conl09e-build\
You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [50]:
import statsmodels.api as sm

C:\Users\latif\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [109]:
def ols(coin):
    X = range(0,len(coin))
    X = sm.add_constant(X)

    model = sm.OLS(coin,X)
    results = model.fit()
    return results.params

### Bitcoin

In [110]:
ols(bitcoin)

array([9660.64956996,  -14.86109699])

### Etherum

In [111]:
ols(etherum)

array([705.01131344,  -1.22019165])

## Nearest Neighbor

In [120]:
from sklearn.neighbors import KNeighborsRegressor
import numpy as np

In [129]:
def nearest_neighbor(coin):
    btc = np.array(coin)
    btc = btc.reshape(-1,1)

    X = np.array(range(0, len(coin)))
    X = X.reshape(-1,1)
    
    neigh = KNeighborsRegressor(n_neighbors=2)
    neigh.fit(X, btc) 
    KNeighborsRegressor(...)
    print(neigh.predict([[1.5]]))

### Bitcoin

In [130]:
nearest_neighbor(bitcoin)

[[8187.4375]]


### Etherum

In [131]:
nearest_neighbor(etherum)

[[685.9525]]


## Analysis

In the case of the two prediction models, OLS predicts a signifcantly higher value for Bitcoin at 9660, and Etherum at a lower value at 705.5 comapred to their current values of 8518 and 714 respectively. K-Nearest Neighbor prediction model predicted lower values for both. 